In [1]:
import pandas as pd

In [17]:
import matplotlib.pyplot as plt

In [2]:
import snorkel

In [3]:
from snorkel.labeling import labeling_function

## Reading the data

In [11]:
#readin the curb_fp.csv file
curb_fp = pd.read_csv('data/curb_fp.csv')
curb_fp

,audit_task_id,label_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,notsure_count,...,canvas_width,canvas_height,canvas_x,canvas_y,zoom,heading,pitch,photographer_heading,photographer_pitch,geometry
0,18927,85882,CurbRamp,1.0,NaN,False,yLFcq-aaf-Ag_iITWsAWnQ,0,0,0,...,720,480,169,141,1,34.671875,-24.8750,90.021843,1.101685,POINT (1275705.9782665605 214304.255489019)
1,19027,86190,CurbRamp,1.0,1.0,False,4vGkAd8TMCx4FgyoDV7Pmw,1,0,0,...,720,480,305,283,1,270.000000,-10.0000,90.147316,-9.087791,POINT (1261597.9105232325 237078.0516430551)
2,19143,86478,CurbRamp,1.0,NaN,False,JXW_FToo2iFIB0NslFrCeg,0,0,0,...,720,480,193,153,1,195.500000,-17.8750,350.722260,2.929657,POINT (1272276.4832222403 258029.08524980946)
3,19411,86767,CurbRamp,1.0,NaN,False,Z1u8iq5TEuS8-n_kZKRc8A,0,0,0,...,720,480,526,315,1,174.437500,-5.8750,156.979935,-0.590858,POINT (1283388.097714087 258754.43556004378)
4,19411,86768,CurbRamp,1.0,1.0,False,Z1u8iq5TEuS8-n_kZKRc8A,1,0,0,...,720,480,285,212,1,128.500000,-13.0000,156.979935,-0.590858,POINT (1283444.4740236972 258754.76484089182)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10760,18558,84922,CurbRamp,NaN,NaN,False,-B8963iDbur4NYIYzdTJXA,0,0,0,...,720,480,505,230,1,324.500000,-17.8125,220.064758,-4.507965,POINT (1269212.1309289748 227124.1054278093)
10761,75784,230293,CurbRamp,1.0,NaN,True,3KIM9a467DImsFzpQ9xAqQ,0,0,0,...,720,480,449,120,3,169.227676,-15.8750,22.459303,2.169060,POINT (1280570.7039149986 206540.29524511957)
10762,76193,231338,CurbRamp,1.0,NaN,True,d3oBrJ-nDtKMwlE7svc6pg,0,0,1,...,720,480,267,187,1,253.562500,-35.0000,177.959915,-0.962440,POINT (1282827.7064719573 208828.7977988457)
10763,76193,231339,CurbRamp,1.0,NaN,True,AOIBJ12BjFanDJF1Ark05Q,0,0,0,...,720,480,516,171,1,110.312500,-35.0000,179.365723,-1.093400,POINT (1282862.9675030417 208900.50316090064)


In [12]:
#put curb_fp[label_id] into a list
curb_fp_ids = curb_fp['label_id'].tolist()

In [23]:
labels = pd.read_csv('data/labels.csv')

In [24]:
labels

,audit_task_id,label_id,label_type,severity,correct,high_quality_user,gsv_panorama_id,agree_count,disagree_count,notsure_count,...,zoom,heading,pitch,photographer_heading,photographer_pitch,geometry,label_tags,label_description,heading_diff,pitch_diff
0,18604,85055,CurbRamp,1.0,NaN,False,NjPAkwTxWsayAq3kCugvdA,0,0,0,...,1,107.750000,-15.62500,270.556610,0.907036,POINT (-122.3327865600586 47.61859893798828),0.0,0.0,-162.806610,-16.532036
1,18608,85057,Obstacle,2.0,1.0,False,f_Mibt-6nJU7HTN7kx5ZGQ,2,0,0,...,1,38.375000,-21.87500,164.133438,0.335564,POINT (-122.3318862915039 47.61780548095703),0.0,0.0,-125.758438,-22.210564
2,18608,85059,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,1,131.187500,-24.50000,164.133438,0.335564,POINT (-122.33183288574219 47.61751174926758),0.0,0.0,-32.945938,-24.835564
3,18608,85060,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,1,131.187500,-24.50000,164.133438,0.335564,POINT (-122.33187866210938 47.61751937866211),0.0,0.0,-32.945938,-24.835564
4,18608,85062,CurbRamp,1.0,NaN,False,f_Mibt-6nJU7HTN7kx5ZGQ,0,0,0,...,1,257.187500,-35.00000,164.133438,0.335564,POINT (-122.33204650878906 47.617637634277344),0.0,0.0,93.054062,-35.335564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195538,76181,231270,CurbRamp,1.0,NaN,True,R-sTl5O6sOdVPurgTWy3EQ,0,0,0,...,1,332.468750,-35.00000,215.130295,-0.157570,POINT (-122.26725769042969 47.540771484375),0.0,0.0,117.338455,-34.842430
195539,76183,231285,NoSidewalk,5.0,NaN,True,N5MrAAtHZO2JGXUrYAYnSQ,0,0,0,...,3,0.890625,-11.65625,179.161057,-5.825714,POINT (-122.26344299316406 47.55229187011719),0.0,0.0,-178.270432,-5.830536
195540,76193,231338,CurbRamp,1.0,NaN,True,d3oBrJ-nDtKMwlE7svc6pg,0,0,1,...,1,253.562500,-35.00000,177.959915,-0.962440,POINT (-122.28179168701172 47.563270568847656),0.0,0.0,75.602585,-34.037560
195541,76193,231339,CurbRamp,1.0,NaN,True,AOIBJ12BjFanDJF1Ark05Q,0,0,0,...,1,110.312500,-35.00000,179.365723,-1.093400,POINT (-122.28165435791016 47.56346893310547),0.0,0.0,-69.053223,-33.906600


## Labeling functions based on heuristics

In [4]:
# Define the label mappings for convenience
NOT_SURE = -1
WRONG = 0
CORRECT = 1

### intersection

In [ ]:
#check if it is in the curb false positive list
@labeling_function()
def intersection(x):
    # x in this case is a row of the dataframe
    return WRONG if x["label_id"] in curb_fp_ids else NOT_SURE

### gsv

In [ ]:
#check if the user zoomed in
@labeling_function()
def zoom(x):
    # x in this case is a row of the dataframe
    return CORRECT if x["zoom"] > 1 else NOT_SURE

In [ ]:
#check if the user changed the heading
@labeling_function()
def heading(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['heading_diff']  != 0 else NOT_SURE

In [ ]:
#check if the user changed the pitch
@labeling_function()
def pitch(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['pitch_diff']  != 0 else NOT_SURE

### optional input

In [ ]:
#check if the user has put a tag
@labeling_function()
def tags(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['label_tags']  == 1 else NOT_SURE

In [ ]:
#check if the user has put a tag
@labeling_function()
def description(x):
    # x in this case is a row of the dataframe
    return CORRECT if x['label_description']  == 1 else NOT_SURE